In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import cvxpy as cp
import plotly.graph_objects as go
from tqdm.auto import tqdm
import statsmodels.api as sm
import warnings
from datetime import datetime, timedelta
from joblib import Parallel, delayed
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import time
import csv
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.timeseries import TimeSeries
import pickle


# print(plt.style.available) #list of available styles
#plt.style.use('ggplot')
# Configura el estilo de Seaborn para que los gráficos se vean más atractivos
sns.set(style="whitegrid")

plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 100
warnings.simplefilter(action='ignore', category=FutureWarning)

Cargar los tickers de los componentes del SP500 de wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
respuesta = requests.get(url)
soup = BeautifulSoup(respuesta.text, 'html.parser')

tabla = soup.find('table', {'id': 'constituents'})
filas = tabla.findAll('tr')

tickers_sp500 = []
for fila in filas[1:]:  # Excluir el encabezado de la tabla
    ticker = fila.findAll('td')[0].text.strip()
    tickers_sp500.append(ticker)

print(tickers_sp500)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DH

In [3]:
with open('../../credentials/alphavantage_key.txt', 'r') as file:
    api_key = file.read().strip()

Otra opción es leerlos desde un fichero

In [19]:
factores_sp500 = pd.read_csv("../data/book_to_market_sp500.csv", index_col=0)
factores_sp500

,Market Capitalization,Book to Market Ratio
A,3.857104e+10,0.151539
AAL,9.149574e+09,-0.633800
AAPL,3.017854e+12,0.020593
ABBV,2.930885e+11,0.058870
ABNB,9.060771e+10,0.061363
...,...,...
YUM,3.673717e+10,-0.241608
ZBH,2.524908e+10,0.476334
ZBRA,1.315268e+10,0.207790
ZION,6.416506e+09,0.762565


In [20]:
threshold = 0.3
# Calcular los percentiles 20 y 80 para la capitalización de mercado
percentil_inf = factores_sp500['Market Capitalization'].quantile(threshold)
percentil_sup = factores_sp500['Market Capitalization'].quantile(1-threshold)

# Asignar valores a la columna SMB
factores_sp500['SMB'] = np.where(factores_sp500['Market Capitalization'] > percentil_sup, 1,
                                 np.where(factores_sp500['Market Capitalization'] < percentil_inf, -1, 0))

In [21]:
# Calcular los percentiles 20 y 80 para la capitalización de mercado
percentil_inf = factores_sp500['Book to Market Ratio'].quantile(threshold)
percentil_sup = factores_sp500['Book to Market Ratio'].quantile(1-threshold)

# Asignar valores a la columna SMB
factores_sp500['HML'] = np.where(factores_sp500['Book to Market Ratio'] > percentil_sup, 1,
                                 np.where(factores_sp500['Book to Market Ratio'] < percentil_inf, -1, 0))

In [23]:
capital_largest = factores_sp500['Market Capitalization'].nlargest(5).index
pb_largest = factores_sp500['Book to Market Ratio'].nlargest(5).index
capital_smallest = factores_sp500['Market Capitalization'].nsmallest(5).index
pb_smallest = factores_sp500['Book to Market Ratio'].nsmallest(5).index

In [116]:
pb_smallest

Index(['AAL', 'DPZ', 'YUM', 'BBWI', 'SBAC'], dtype='object')

In [117]:
pb_largest

Index(['PARA', 'IVZ', 'C', 'WBD', 'CFG'], dtype='object')

In [89]:
capital_smallest

Index(['WHR', 'MHK', 'ZION', 'VFC', 'HAS'], dtype='object')

In [90]:
capital_smb

Index(['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'AMZN', 'WHR', 'MHK', 'ZION', 'VFC',
       'HAS'],
      dtype='object')

## ANÁLISIS DE ACTIVOS MEDIANTE EL MODELO FAMA FRENCH DE TRES FACTORES

El modelo de Fama-French de tres factores amplía el modelo de valoración de activos de capital (CAPM) al incluir dos factores adicionales, tamaño y valor, para explicar mejor los rendimientos de las acciones.

Estos factores ayudan a explicar los rendimientos de las acciones más allá del CAPM, considerando el impacto del tamaño de la empresa y la propensión hacia acciones de valor o de crecimiento.

El modelo se utiliza para analizar y predecir el comportamiento de un activo de la siguiente manera:

- **Incluir el Mercado:** Evalúa cómo el activo reacciona ante los movimientos del mercado general, considerando la diferencia entre los rendimientos del mercado y la tasa libre de riesgo.

- **Considerar el Tamaño:** Analiza el impacto del tamaño de la empresa en el rendimiento del activo, comparando pequeñas empresas con grandes empresas.

- **Evaluar el Valor:** Examina la relación entre el valor de mercado de la empresa y su valor contable, diferenciando entre acciones de crecimiento y acciones de valor.

Utilizando estos tres factores, se puede obtener una comprensión más detallada de las fuerzas que influyen en el rendimiento de un activo, más allá de simplemente mirar el mercado en general. Esto ayuda a identificar si un activo está generando rendimientos por encima o por debajo de lo esperado dadas estas dimensiones de riesgo.


### Los **factores** que utiliza son::

1. **Mkt-RF (Mercado Menos Tasa Libre de Riesgo):** Mide el exceso de rendimiento del mercado sobre la tasa libre de riesgo.

   - Una **beta alta** indica que el activo es más sensible a los movimientos del mercado, mientras que una **beta baja** sugiere menor sensibilidad.

2. **SMB (Small Minus Big):** Representa la prima de riesgo asociada al tamaño de las empresas, comparando el rendimiento de empresas pequeñas con el de empresas grandes.

   - Una **beta positiva** indica una inclinación hacia empresas pequeñas (con mayor riesgo y potencial de rendimiento), y una **beta negativa** sugiere preferencia por empresas grandes y más estables.

3. **HML (High Minus Low):** Mide la diferencia de rendimiento entre empresas de alto valor (valor contable alto en relación con el precio de mercado) y empresas de crecimiento (precio de mercado alto en relación con el valor contable).

   - Una **beta positiva** implica una tendencia hacia empresas de valor, consideradas menos riesgosas pero con potencial de rendimientos consistentes, mientras que una **beta negativa** muestra una tendencia hacia empresas de crecimiento, que pueden ser más riesgosas pero con la posibilidad de altos rendimientos.

Interpretar las betas de estos factores ayuda a comprender cómo diferentes aspectos del mercado y características de la empresa afectan el rendimiento esperado de un activo, permitiendo a los inversores hacer decisiones más informadas basadas en el perfil de riesgo y retorno.


###  A continuación, se describe **cómo se calculan** estos tres factores de manera sencilla:

 1. **Exceso de Rendimiento del Mercado (Mkt-RF):**
Este factor mide la diferencia entre el rendimiento del mercado de acciones y la tasa libre de riesgo. Se calcula restando el rendimiento de un activo libre de riesgo, como los bonos del Tesoro de EE. UU., del rendimiento de un índice bursátil amplio, como el S&P 500.

 2. **Tamaño de la Empresa (SMB, Small Minus Big):**
El factor SMB mide la diferencia en rendimientos entre empresas pequeñas y grandes. Las acciones se dividen en dos grupos basados en su capitalización de mercado: pequeñas y grandes. Luego, se calcula el rendimiento promedio de las acciones pequeñas y se resta del rendimiento promedio de las acciones grandes.

 3. **Valor de la Empresa (HML, High Minus Low):**
El factor HML evalúa la diferencia en rendimientos entre empresas de "alto valor" (value stocks) y de "crecimiento" (growth stocks). Las acciones se clasifican en estas categorías según su relación price to book ratio. Se calcula el rendimiento promedio de las acciones de alto valor y se resta del rendimiento promedio de las acciones de crecimiento.

### Descarga de los Valores de los Factores

<p align="center">
  <img src="../imgs/fama-french-web.PNG" alt="Fama French Web" style="max-width:70%;">
</p>


Los valores de los factores de Fama-French pueden descargarse desde la **Base de Datos de Kenneth R. French**, alojada en la página web de la Tuck School of Business en Dartmouth College. Aquí se ofrecen datos históricos de los factores de riesgo para el mercado de valores de EE.UU. y otros países.

- **URL para la descarga:** [Base de Datos de Kenneth R. French](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)

En esta página, puedes encontrar los factores para diferentes periodos y frecuencias temporales, incluyendo datos diarios, mensuales y anuales.

## Análisis de la acción de Microsoft

Pasos a seguir:

1- Bajarnos los precios de cierre ajustados mensulaes de microsoft y calcular los retornos

2- Bajarnos los factores del modelo de Fama French también mensuales

3- Hacer la regresión lineal según el modelo de fama french: $\quad (R_{activo} - R_f) = \alpha + \beta_1 (Mkt-R_f) + \beta_2 SMB + \beta_3 HML$

4- Analizar los resultados

**1- Nos bajamos los precios y calculamos los retornos.**

Bajamos los precios mensuales ya que os retornos diarios contienen mucho ruido

In [137]:
precios_msft = yf.download('MSFT', start='2015-01-01', end='2023-12-31', interval='1mo')['Adj Close']

returns_msft = precios_msft.pct_change().dropna()

returns_msft

[*********************100%%**********************]  1 of 1 completed


Date
2015-02-01    0.085396
2015-03-01   -0.066149
2015-04-01    0.196262
2015-05-01   -0.036595
2015-06-01   -0.051709
                ...   
2023-08-01   -0.024291
2023-09-01   -0.034603
2023-10-01    0.070815
2023-11-01    0.120671
2023-12-01   -0.005560
Name: Adj Close, Length: 107, dtype: float64

**2- Nos bajamos los factores de un fichero ya descargado y limpio**

In [168]:
ff_factors = pd.read_csv('../data/F-F_Research_Data_Factors_monthly.csv', index_col=0, parse_dates=True)

ff_factors

,Mkt-RF,SMB,HML,RF
1926-07-01,2.96,-2.56,-2.43,0.22
1926-08-01,2.64,-1.17,3.82,0.25
1926-09-01,0.36,-1.40,0.13,0.23
1926-10-01,-3.24,-0.09,0.70,0.32
1926-11-01,2.53,-0.10,-0.51,0.31
...,...,...,...,...
2023-07-01,3.21,2.08,4.11,0.45
2023-08-01,-2.39,-3.16,-1.06,0.45
2023-09-01,-5.24,-2.51,1.52,0.43
2023-10-01,-3.19,-3.87,0.19,0.47


In [169]:
# Dividimos los factores por 100 para poder comparar con los retornos de las acciones
ff_factors = ff_factors / 100
ff_factors

,Mkt-RF,SMB,HML,RF
1926-07-01,0.0296,-0.0256,-0.0243,0.0022
1926-08-01,0.0264,-0.0117,0.0382,0.0025
1926-09-01,0.0036,-0.0140,0.0013,0.0023
1926-10-01,-0.0324,-0.0009,0.0070,0.0032
1926-11-01,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2023-07-01,0.0321,0.0208,0.0411,0.0045
2023-08-01,-0.0239,-0.0316,-0.0106,0.0045
2023-09-01,-0.0524,-0.0251,0.0152,0.0043
2023-10-01,-0.0319,-0.0387,0.0019,0.0047


In [140]:
# Creamos el dataframe con os factores y los retornos de Microsoft

datos_df = pd.concat([ff_factors, returns_msft], axis=1, join='inner')
datos_df.columns = list(ff_factors.columns) + ['MSFT']
datos_df.head(5)

,Mkt-RF,SMB,HML,RF,MSFT
2015-02-01,0.0613,0.0063,-0.0186,0.0,0.085396
2015-03-01,-0.0112,0.0304,-0.0037,0.0,-0.066149
2015-04-01,0.0059,-0.0306,0.0182,0.0,0.196262
2015-05-01,0.0136,0.0093,-0.0114,0.0,-0.036595
2015-06-01,-0.0153,0.0291,-0.0079,0.0,-0.051709


**3- Realizamos la regresión lineal**

In [141]:
Y = datos_df.loc[:, 'MSFT']-datos_df['RF']
X = datos_df[['Mkt-RF', 'SMB', 'HML']]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     56.27
Date:                Sun, 04 Feb 2024   Prob (F-statistic):           1.52e-21
Time:                        19:37:31   Log-Likelihood:                 195.02
No. Observations:                 106   AIC:                            -382.0
Df Residuals:                     102   BIC:                            -371.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0127      0.004      3.254      0.0

**4- Analizar los resultados**

La interpretación de los resultados para la acción de Microsoft (MSFT) en el modelo de tres factores de Fama-French, incluyendo el valor de R-cuadrado, se presenta de manera concisa:

##### **R-cuadrado: 0.623**
El **R-cuadrado** de **0.623** indica que aproximadamente el **62.3%** de la variabilidad en los retornos de Microsoft se explica por los tres factores del modelo de Fama-French. Esto muestra que una buena parte de los movimientos de precio de MSFT puede ser explicada por factores del mercado.

##### **Alpha (constante): 0.0127**
Microsoft ha generado, en promedio, un **rendimiento mensual del 1.27%** por encima de la tasa libre de riesgo, ajustado por los factores de riesgo del modelo. Este exceso de retorno es **estadísticamente significativo** (P = 0.002), indicando un rendimiento superior debido a una buena gestión de la empresa.

##### **Mkt-RF: 1.0261**
La **sensibilidad de Microsoft al mercado** es ligeramente superior a 1, indicando que sus acciones tienden a moverse en línea con el mercado pero con una volatilidad ligeramente mayor. Este efecto es **altamente significativo** (P = 0.000).

##### **SMB: -0.7056**
Microsoft muestra una **beta negativa para SMB**, reflejando su estatus como una gran corporación y su tendencia a rendir de manera opuesta a las pequeñas empresas. La **significancia estadística** (P = 0.000) confirma la relevancia de este factor.

##### **HML: -0.4315**
Una **beta negativa para HML** sugiere que Microsoft actúa más como una acción de crecimiento que de valor. Este efecto también es **estadísticamente significativo** (P = 0.000).

En resumen, el análisis muestra que **Microsoft ha superado el rendimiento esperado** basado en los riesgos del mercado y factores específicos, con su rendimiento en gran medida explicado por el modelo de Fama-French.


---

In [164]:
# Definir una función para analizar las fechas en el formato específico
dateparse = lambda x: pd.to_datetime(x, format='%Y%m')

# Leer el archivo CSV con el analizador de fechas personalizado
industries_ff_df = pd.read_csv('../data/5_Industry_Portfolios.csv', 
                               index_col=0, 
                               parse_dates=True, 
                               date_parser=dateparse)

In [170]:
industries_ff_df = industries_ff_df / 100
industries_ff_df

,Cnsmr,Manuf,HiTec,Hlth,Other
1926-07-01,0.0543,0.0273,0.0183,0.0177,0.0213
1926-08-01,0.0276,0.0233,0.0241,0.0425,0.0435
1926-09-01,0.0216,-0.0044,0.0106,0.0069,0.0029
1926-10-01,-0.0390,-0.0242,-0.0226,-0.0057,-0.0284
1926-11-01,0.0370,0.0250,0.0307,0.0542,0.0211
...,...,...,...,...,...
2023-08-01,-0.0182,-0.0182,-0.0158,-0.0022,-0.0335
2023-09-01,-0.0494,-0.0395,-0.0582,-0.0471,-0.0341
2023-10-01,-0.0343,-0.0305,-0.0168,-0.0458,-0.0253
2023-11-01,0.0788,0.0522,0.1165,0.0587,0.1041


In [172]:
#industries_ff_df['RF'] = ff_factors['RF']
datos_ind_df = pd.concat([industries_ff_df, returns_msft], axis=1, join='inner')
datos_ind_df.columns = list(industries_ff_df.columns) + ['MSFT']
datos_ind_df = pd.concat([datos_ind_df,ff_factors['RF']], axis=1, join='inner')
datos_ind_df.head(5)

,Cnsmr,Manuf,HiTec,Hlth,Other,MSFT
2015-02-01,0.0562,0.0370,0.0839,0.0424,0.0721,0.085396
2015-03-01,-0.0050,-0.0188,-0.0239,0.0076,-0.0059,-0.066149
2015-04-01,-0.0105,0.0166,0.0190,-0.0142,0.0062,0.196262
2015-05-01,0.0114,-0.0122,0.0164,0.0491,0.0174,-0.036595
2015-06-01,-0.0092,-0.0310,-0.0293,0.0016,0.0003,-0.051709


In [177]:
Y = datos_ind_df.loc[:, 'MSFT']-datos_ind_df['RF']
X = datos_ind_df[['Cnsmr', 'Manuf', 'HiTec', 'Hlth ', 'Other']]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     39.88
Date:                Mon, 05 Feb 2024   Prob (F-statistic):           2.34e-22
Time:                        09:18:00   Log-Likelihood:                 201.39
No. Observations:                 106   AIC:                            -390.8
Df Residuals:                     100   BIC:                            -374.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0092      0.004      2.434      0.0

---

https://www.blackrock.com/ch/individual/en/products/228590/blackrock-us-basic-value-e2-eur-fund

In [194]:
fund_data_df = pd.read_excel('../data/BGF-US-Basic-Value-Fund-Class-E2-EUR_fund.xlsx', sheet_name='Performance', skiprows=4, index_col=0, parse_dates=True)
fund_data_df.columns = ['Return']
fund_data_df.head(5)

,Return
Month End Date,
2003-04-30,-0.859290
2003-05-31,-0.938970
2003-06-30,6.270504
2003-07-31,1.749575
2003-08-31,7.585979


In [193]:
ff_factors = pd.read_csv('../data/F-F_Research_Data_Factors_monthly.csv', index_col=0, parse_dates=True)
#ff_factors = ff_factors / 100

ff_factors

,Mkt-RF,SMB,HML,RF
1926-07-01,2.96,-2.56,-2.43,0.22
1926-08-01,2.64,-1.17,3.82,0.25
1926-09-01,0.36,-1.40,0.13,0.23
1926-10-01,-3.24,-0.09,0.70,0.32
1926-11-01,2.53,-0.10,-0.51,0.31
...,...,...,...,...
2023-07-01,3.21,2.08,4.11,0.45
2023-08-01,-2.39,-3.16,-1.06,0.45
2023-09-01,-5.24,-2.51,1.52,0.43
2023-10-01,-3.19,-3.87,0.19,0.47


In [195]:
# Creamos el dataframe con os factores y los retornos de Microsoft

datos_df = pd.concat([ff_factors, returns_msft], axis=1, join='inner')
datos_df.columns = list(ff_factors.columns) + ['Return']
datos_df.head(5)

,Mkt-RF,SMB,HML,RF,Return
2015-02-01,6.13,0.63,-1.86,0.0,0.085396
2015-03-01,-1.12,3.04,-0.37,0.0,-0.066149
2015-04-01,0.59,-3.06,1.82,0.0,0.196262
2015-05-01,1.36,0.93,-1.14,0.0,-0.036595
2015-06-01,-1.53,2.91,-0.79,0.0,-0.051709


In [196]:
Y = datos_df.loc[:, 'Return']-datos_df['RF']
X = datos_df[['Mkt-RF', 'SMB', 'HML']]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     5.019
Date:                Mon, 05 Feb 2024   Prob (F-statistic):            0.00276
Time:                        17:17:07   Log-Likelihood:                 65.924
No. Observations:                 106   AIC:                            -123.8
Df Residuals:                     102   BIC:                            -113.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0913      0.013     -6.924      0.0

---

In [25]:
capital_smb = capital_largest.append(capital_smallest)
pb_hml = pb_largest.append(pb_smallest)

In [105]:
precios_smb = yf.download(list(capital_smb), start="2015-01-01", end="2023-12-31", interval="1mo")['Adj Close']
returns_smb = precios_smb.pct_change().dropna()
precios_hml = yf.download(list(pb_hml), start="2015-01-01", end="2023-12-31", interval="1mo")['Adj Close']
returns_hml = precios_hml.pct_change().dropna()

[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed


In [80]:
ff_factors = pd.read_csv('../data/F-F_Research_Data_Factors_monthly.csv', index_col=0, parse_dates=True)
ff_factors

,Mkt-RF,SMB,HML,RF
1926-07-01,2.96,-2.56,-2.43,0.22
1926-08-01,2.64,-1.17,3.82,0.25
1926-09-01,0.36,-1.40,0.13,0.23
1926-10-01,-3.24,-0.09,0.70,0.32
1926-11-01,2.53,-0.10,-0.51,0.31
...,...,...,...,...
2023-07-01,3.21,2.08,4.11,0.45
2023-08-01,-2.39,-3.16,-1.06,0.45
2023-09-01,-5.24,-2.51,1.52,0.43
2023-10-01,-3.19,-3.87,0.19,0.47


In [81]:
ff_factors = ff_factors / 100
ff_factors

,Mkt-RF,SMB,HML,RF
1926-07-01,0.0296,-0.0256,-0.0243,0.0022
1926-08-01,0.0264,-0.0117,0.0382,0.0025
1926-09-01,0.0036,-0.0140,0.0013,0.0023
1926-10-01,-0.0324,-0.0009,0.0070,0.0032
1926-11-01,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2023-07-01,0.0321,0.0208,0.0411,0.0045
2023-08-01,-0.0239,-0.0316,-0.0106,0.0045
2023-09-01,-0.0524,-0.0251,0.0152,0.0043
2023-10-01,-0.0319,-0.0387,0.0019,0.0047


In [82]:
datos_smb_df = pd.concat([ff_factors, returns_smb], axis=1, join='inner')
datos_smb_df.head(5)

,Mkt-RF,SMB,HML,RF,AAPL,AMZN,GOOG,GOOGL,HAS,MHK,MSFT,VFC,WHR,ZION
2015-02-01,0.0613,0.0063,-0.0186,0.0,0.096449,0.072293,0.044676,0.046656,0.143559,0.117002,0.085396,0.105088,0.064648,0.116027
2015-03-01,-0.0112,0.0304,-0.0037,0.0,-0.027549,-0.021202,-0.018625,-0.014095,0.014762,0.007594,-0.066149,-0.017610,-0.043314,0.011260
2015-04-01,0.0059,-0.0306,0.0182,0.0,0.005786,0.133513,-0.016760,-0.010691,0.119387,-0.065949,0.196262,-0.034179,-0.130951,0.049630
2015-05-01,0.0136,0.0093,-0.0114,0.0,0.040991,0.017663,-0.009733,-0.006287,0.025563,0.075735,-0.036595,-0.027613,0.049260,0.019054
2015-06-01,-0.0153,0.0291,-0.0079,0.0,-0.033206,0.011323,-0.021800,-0.009682,0.036878,0.022825,-0.051709,-0.009797,-0.056232,0.101296


In [115]:
Y = datos_smb_df.loc[:, 'VFC']-datos_smb_df['RF']

X = datos_smb_df[['Mkt-RF', 'SMB', 'HML']]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     35.00
Date:                Sun, 04 Feb 2024   Prob (F-statistic):           1.23e-15
Time:                        11:46:56   Log-Likelihood:                 140.84
No. Observations:                 106   AIC:                            -273.7
Df Residuals:                     102   BIC:                            -263.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0196      0.007     -3.013      0.0

In [109]:
resultados_smb = {}
for i in capital_smb:
    Y = datos_smb_df.loc[:, i]-datos_smb_df['RF']
    X = datos_smb_df[['Mkt-RF', 'SMB', 'HML']]
    X = sm.add_constant(X)
    modelo = sm.OLS(Y, X).fit()
    resultados_smb[i] = pd.concat([modelo.params,modelo.pvalues])
    
resultados_smb_df = pd.concat(resultados_smb, axis=1).T

resultados_smb_df.columns = ['alpha', 'Mkt-RF', 'SMB', 'HML', 'alpha_p', 'Mkt-RF_p', 'SMB_p', 'HML_p']
resultados_smb_df

,alpha,Mkt-RF,SMB,HML,alpha_p,Mkt-RF_p,SMB_p,HML_p
AAPL,0.008267,1.277976,-0.283475,-0.535225,0.134873,1.544909e-18,1.777782e-01,2.167061e-04
MSFT,0.012688,1.026095,-0.705640,-0.431471,0.001546,1.364653e-21,6.635864e-06,3.236136e-05
GOOGL,0.005430,1.102575,-0.444106,-0.344488,0.271512,1.138312e-17,1.944847e-02,6.899949e-03
GOOG,0.005521,1.109951,-0.460992,-0.342371,0.259484,5.036250e-18,1.454081e-02,6.767230e-03
AMZN,0.009205,1.318192,-0.324353,-0.948770,0.125606,2.498999e-17,1.560801e-01,9.271903e-09
WHR,-0.011170,1.430480,0.158267,0.498579,0.128146,8.465626e-15,5.694596e-01,8.306762e-03
MHK,-0.011276,1.195283,0.694102,0.417513,0.127609,2.061744e-11,1.465885e-02,2.751284e-02
ZION,0.004636,0.999200,1.350464,1.274699,0.455826,2.900710e-11,1.043352e-07,1.290348e-12
VFC,-0.019590,1.388359,-0.180867,0.230391,0.003264,1.509566e-16,4.667897e-01,1.666080e-01
HAS,-0.003781,0.775531,-0.282058,0.198347,0.606865,3.645882e-06,3.143729e-01,2.895910e-01


In [110]:
factores_sp500.loc[capital_smb]

,Market Capitalization,Book to Market Ratio,SMB,HML
AAPL,3.017854e+12,0.020593,1,-1
MSFT,2.964729e+12,0.069559,1,-1
GOOGL,1.849748e+12,0.138475,1,0
GOOG,1.837055e+12,0.139432,1,0
AMZN,1.612311e+12,0.090580,1,-1
WHR,6.119400e+09,0.381737,-1,0
MHK,6.238308e+09,1.285271,-1,1
ZION,6.416506e+09,0.762565,-1,1
VFC,6.502124e+09,0.447656,-1,0
HAS,6.713402e+09,0.426297,-1,0


In [106]:
datos_hml_df = pd.concat([ff_factors, returns_hml], axis=1, join='inner')
datos_hml_df.head(5)

,Mkt-RF,SMB,HML,RF,AAL,BBWI,C,CFG,DPZ,IVZ,PARA,SBAC,WBD,YUM
2015-02-01,0.0613,0.0063,-0.0186,0.0,-0.024043,0.085431,0.116744,0.039330,0.025038,0.096379,0.078270,0.068638,0.114177,0.128536
2015-03-01,-0.0112,0.0304,-0.0037,0.0,0.104111,0.054818,-0.017169,-0.024518,-0.009652,-0.008128,0.025888,-0.061022,-0.047678,-0.029466
2015-04-01,0.0059,-0.0306,0.0182,0.0,-0.085070,-0.052285,0.034938,0.079569,0.075968,0.043588,0.027283,-0.010931,0.052016,0.091972
2015-05-01,0.0136,0.0093,-0.0114,0.0,-0.120794,-0.031782,0.014254,0.030326,0.007511,-0.038387,-0.006599,-0.034623,0.048826,0.053797
2015-06-01,-0.0153,0.0291,-0.0079,0.0,-0.057352,-0.009131,0.022417,0.021433,0.043622,-0.052483,-0.100778,0.028262,-0.020035,-0.000333


In [107]:
resultados_hml = {}
for i in returns_hml:
    Y = datos_hml_df.loc[:, i]-datos_hml_df['RF']
    X = datos_hml_df[['Mkt-RF', 'SMB', 'HML']]
    X = sm.add_constant(X)
    modelo = sm.OLS(Y, X).fit()
    resultados_hml[i] = pd.concat([modelo.params,modelo.pvalues])
    
resultados_hml_df = pd.concat(resultados_hml, axis=1).T

resultados_hml_df.columns = ['alpha', 'Mkt-RF', 'SMB', 'HML', 'alpha_p', 'Mkt-RF_p', 'SMB_p', 'HML_p']
resultados_hml_df

,alpha,Mkt-RF,SMB,HML,alpha_p,Mkt-RF_p,SMB_p,HML_p
AAL,-0.016459,1.358163,0.710875,0.862782,0.068053,2.282780e-10,3.896150e-02,2.456351e-04
BBWI,-0.005657,1.442455,0.073087,0.534550,0.677881,3.415818e-06,8.879014e-01,1.248421e-01
C,-0.006088,1.463736,0.288729,0.984836,0.229893,2.663475e-24,1.355988e-01,9.891339e-12
CFG,0.001108,1.206886,1.030921,1.436903,0.829492,9.354796e-19,7.528000e-07,4.885993e-19
DPZ,0.008519,0.782140,-0.204459,-0.390194,0.273151,8.852448e-06,4.890486e-01,4.994138e-02
IVZ,-0.010601,1.300845,0.518509,0.906285,0.165085,2.780826e-12,7.547872e-02,8.170509e-06
PARA,-0.014983,1.593241,0.740864,0.767882,0.147613,1.171203e-10,6.098505e-02,4.059673e-03
SBAC,0.000448,0.719359,-0.858143,-0.509189,0.938044,7.893951e-08,1.611105e-04,7.348171e-04
WBD,-0.013056,1.353288,0.975950,0.434842,0.227027,6.591454e-08,1.889416e-02,1.145897e-01
YUM,0.002217,0.941018,-0.416517,0.163140,0.667625,2.068669e-13,3.604577e-02,2.156584e-01


In [108]:
factores_sp500.loc[pb_hml]

,Market Capitalization,Book to Market Ratio,SMB,HML
PARA,9.188025e+09,2.507176,-1,1
IVZ,7.817743e+09,2.154356,-1,1
C,1.014352e+11,1.993016,1,1
WBD,2.570253e+10,1.893471,0,1
CFG,1.551306e+10,1.527100,-1,1
AAL,9.149574e+09,-0.633800,-1,-1
DPZ,1.492942e+10,-0.280591,-1,-1
YUM,3.673717e+10,-0.241608,0,-1
BBWI,9.362994e+09,-0.235608,-1,-1
SBAC,2.522290e+10,-0.209187,0,-1
